In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [5]:
# Hyper-parameters 
input_size = 28 * 28
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [11]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
# MNIST datasets (images and labels)
train_dataset = torchvision.datasets.MNIST(root='/data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='/data', train=False, transform=transforms.ToTensor())

# Data loaders (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# NN Module Template
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out  

In [12]:
# Instantiate NN
basic_nn = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(basic_nn.parameters(), lr=learning_rate)

In [15]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = basic_nn(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0346
Epoch [1/5], Step [200/600], Loss: 0.0085
Epoch [1/5], Step [300/600], Loss: 0.0380
Epoch [1/5], Step [400/600], Loss: 0.0118
Epoch [1/5], Step [500/600], Loss: 0.0209
Epoch [1/5], Step [600/600], Loss: 0.0300
Epoch [2/5], Step [100/600], Loss: 0.0235
Epoch [2/5], Step [200/600], Loss: 0.0169
Epoch [2/5], Step [300/600], Loss: 0.0218
Epoch [2/5], Step [400/600], Loss: 0.0860
Epoch [2/5], Step [500/600], Loss: 0.0212
Epoch [2/5], Step [600/600], Loss: 0.0185
Epoch [3/5], Step [100/600], Loss: 0.0050
Epoch [3/5], Step [200/600], Loss: 0.0116
Epoch [3/5], Step [300/600], Loss: 0.0026
Epoch [3/5], Step [400/600], Loss: 0.0063
Epoch [3/5], Step [500/600], Loss: 0.0559
Epoch [3/5], Step [600/600], Loss: 0.0082
Epoch [4/5], Step [100/600], Loss: 0.0165
Epoch [4/5], Step [200/600], Loss: 0.0084
Epoch [4/5], Step [300/600], Loss: 0.0082
Epoch [4/5], Step [400/600], Loss: 0.0067
Epoch [4/5], Step [500/600], Loss: 0.0173
Epoch [4/5], Step [600/600], Loss:

In [17]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        
        outputs = basic_nn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 98 %


### 98% on a two layer NN!!